In [1]:
#Imports 

import sys
import os
import yaml

sys.path.append(os.getenv("CODE_PATH"))
sys.path.append(os.getenv("FIN_DATABASE_PATH"))


import plotly.graph_objects as go
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import logging
import matplotlib.pyplot as plt
from dotenv import load_dotenv
from Data.connect import engine, DailyStockData, HourlyStockData, OneMinuteStockData, FiveMinuteStockData,FifteenMinuteStockData, StockSplits, StockNews, CompanyFinancials
from Pre_Processing.pre_processing import PreProcessing
from data_fetcher import DataFetcher
from Feature_Engineering.feature_engineering import TechnicalIndicators
from pipeline import Pipeline
import json
from pandas import json_normalize
from fs_pre_processing import PreProcessingFinancials
from metrics import CalculateMetrics

In [14]:
import openai

In [16]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
GENERAL_COLUMNS = ['company_name', 'start_date', 'end_date', 'filing_date',
       'fiscal_period', 'fiscal_year', 'acceptance_datetime',
       'timeframe', 'tickers', 'sic']
SECTIONS = ['balance_sheet', 'income_statement', 'cash_flow_statement', 'comprehensive_income']


In [ ]:
# class PreProcessingFinancials:
#     def __init__(self, data, sections, tickers):
#         """
#         args: 
#         data: dictionary with multiple tickers
#         sections: list with sections from a Financial Statement
#         tickers: list with tickers we want to analyse
#         """
#         if isinstance(data,pd.DataFrame): #If we pass only one ticker, which would be only one dataframe, we transform into a dictionary.
#             self.data = {}
#             self.data[tickers] = data
#             self.tickers = [tickers]
#         else:
#             self.data = data
#             self.tickers = tickers 
#         self.sections = sections
        
#     def adjust_data(self, data):
#         """Adjusting filling date for TTM, Q4, FY, as they don't have any filing dates.
#         Using as a proxy the end date of the period plus 37 days, which is the average time it takes to file the 10-K/10-Q."""
#         conditions = (
#             (data['filing_date'].isna()) &
#             (data['fiscal_period'].isin(['TTM', 'Q4', 'FY']))
#                         )
    
#         data['filing_date'] = np.where(conditions, data['end_date'] + timedelta(days=37), data['filing_date'])
      
#         return data
    
#     def replacing_nan(self,data):
#         """Replace NaN values in the fiscal year column with the correct date."""
#         data['period'] = np.where(data['fiscal_'])
        
    
#     def flatten_json_section(self):
#         """Preprocess the financials column by flattening JSON fields and handling filing dates/fiscal periods."""
#         for ticker in self.tickers:
#             data = self.data[ticker]
            
#             # Converting JSON strings into Python dictionaries if necessary
#             data['financials'] = data['financials'].apply(
#                 lambda x: json.loads(x) if isinstance(x, str) else x
#             )

#             # Flattening each section and concatenate with the main dataframe
#             for section in self.sections:
#                 flattened = self.flattening(data, 'financials', section)
#                 data = pd.concat([data, flattened], axis=1)

#             # Handle filing dates and fiscal periods
#             data['filing_date'] = pd.to_datetime(data['filing_date'])

#             #Adjust filing dates before setting index
#             data = self.adjust_data(data)

#             #Sort by filing date 
#             data.sort_values(by='filing_date', inplace=True, ascending=False)

#             self.data[ticker] = data
    
    
    
#     def flattening(self, data, json_col, section):
#         """ Helper function to flatten a JSON section of the financials dataframe
#         """
#         section_data = data[json_col].apply(lambda x: x.get(section) if section in x else {})
#         flattened_section = json_normalize(section_data)  # Flatten the section
#         flattened_section.columns = [f"{section}_{col}" for col in flattened_section.columns]  # Add prefix to columns
                
#         return flattened_section

    
#     def removing_cols(self):
#         """ This function cleans the dataframe by dropping columns with '.unit' in the name and '.order' in the name.
#         If .unit columns are the same for each row, we will drop.
#         Also dropping columns that have .order
#         """
        
#         for ticker in self.tickers:
#             data = self.data[ticker]
#             # print(f'Processing ticker {ticker}')
#             for section in self.sections:
#                 section_columns = [col for col in data.columns if section in col]
#                 section_df = data[section_columns].copy()
            
#                 #Removing .order columns
#                 order_columns = [col for col in section_df.columns if '.order' in col]
#                 # if order_columns:
#                 #     print(f"Found '.order' columns for {ticker} in {section}: {order_columns}")  # Debugging
#                 # else:
#                 #     print(f"No '.order' columns found for {ticker} in {section}") 
#                 section_df.drop(columns=order_columns, inplace=True)
                
#                 #Removing .unit columns if they only have one unique value
#                 unit_columns = [col for col in section_df.columns if '.unit' in col]
#                 # print(f"Found '.unit' columns for {ticker} in {section}: {unit_columns}")  # Debugging
 
#                 for col in unit_columns:
#                     if section_df[col].nunique() == 1:
#                         section_df.drop(columns=col, inplace=True)
        
                
#                 label_columns = [col for col in section_df.columns if '.label' in col]
#                 section_df.drop(columns=label_columns, inplace=True)
                
                
#                 #Converting numeric values to millions
#                 value_columns = [col for col in section_df.columns if '.value' in col]
#                 for col in value_columns:
#                     section_df[col] = pd.to_numeric(section_df[col])
#                     section_df[col] = section_df[col]/1000000
#                     # print(f"Converted {col} to millions for {ticker} in {section}")  # Debugging
#                 data.drop(columns=section_columns, inplace=True)  # Remove the original section columns
#                 data = pd.concat([data, section_df], axis=1)
               
#             self.data[ticker] = data
    
#     def transform_columns(self, data):
#         """Transform data column data into required type"""
        
#         # Replacing empty strings with np.nan in the entire dataframe at once
#         data.replace('', np.nan, inplace=True)
        
#         # Convert specific columns to required types
#         data['fiscal_year'] = pd.to_numeric(data['fiscal_year'], errors='coerce')
#         data['fiscal_period'] = data['fiscal_period'].astype('category')
#         data['start_date'] = pd.to_datetime(data['start_date'], errors='coerce')
#         data['end_date'] = pd.to_datetime(data['end_date'], errors='coerce')
        
#     def create_period(self, row):
#         if row['timeframe'] == 'quarterly':
#             quarter = int(row['fiscal_period'].replace('Q', ''))
#             year = int(row['fiscal_year'])
#             return pd.Period(freq='Q', year=year, quarter=quarter)
#         else:
#             return np.nan
        
        
#     def preprocess_financials(self):
#         """Orchestrates the entire pre-processing of financials."""
#         self.flatten_json_section()  
#         self.removing_cols() 
        
#         # Step 3: Create multi-indexed DataFrame for each ticker
#         processed_data = {}
        
#         for ticker in self.tickers:
#             data = self.data[ticker]
            
#             # Extract general columns from the data
#             general_df = data[GENERAL_COLUMNS].copy()

#             # If 'filing_date' is not already an index, set it as the index in general_df
#             if 'filing_date' in general_df.columns:
#                 general_df.set_index('filing_date', inplace=True)
            
#             # Proccessing each section
#             section_dataframes = []
            
           
#             for section in self.sections:
#                 # Filter columns related to the current section
#                 section_columns = [col for col in data.columns if section in col]
#                 section_df = data[['filing_date'] + section_columns].copy()  # Ensure 'filing_date' is included

#                 # Set 'filing_date' as the index for the section to align it properly
#                 section_df.set_index('filing_date', inplace=True)

#                 # Removing '.value' suffix from the column names
#                 section_df.columns = section_df.columns.str.replace('.value', '', regex=False)

#                 # Removing the section name from the second-level column names
#                 section_df.columns = pd.MultiIndex.from_product(
#                     [[section], section_df.columns.str.replace(f'{section}_', '', regex=False)]
#                 )

#                 # Add this section DataFrame to the list
#                 section_dataframes.append(section_df)
            
#             # Concatenate all section DataFrames into one DataFrame (financial data)
#             financial_data = pd.concat(section_dataframes, axis=1)

#             # Concatenate general_df (with general columns) and financial_data (with sections)
#             full_data = pd.concat([general_df, financial_data], axis=1)

#             # Store the processed DataFrame
#             processed_data[ticker] = full_data
            
#         combined_data = pd.concat(processed_data.values(), keys=processed_data.keys(), names=['ticker'])
#         self.transform_columns(combined_data)
#         combined_data['period'] = combined_data.apply(self.create_period, axis=1)
        
        
#         return processed_data, combined_data

In [ ]:
# class CalculateMetrics:
    
#     def __init__(self, data):
#         """ data is a dataframe with tickers as index."""
#         self.data = data
        
#     def profitability_ratios(self):
#         """Calculates profitability ratios."""
#         # data = self.self.data.copy()
        
#         # Gross Margin
#         self.data['gross_margin'] = self.data[('income_statement', 'gross_profit')] / self.data[('income_statement', 'revenues')]
        
#         # Operating Margin
#         self.data['operating_margin'] = self.data[('income_statement', 'operating_income_loss')] / self.data[('income_statement', 'revenues')]
        
#         # Net Profit Margin
#         self.data['net_profit_margin'] = self.data[('income_statement', 'net_income_loss')] / self.data[('income_statement', 'revenues')]
        
#         #ROA
#         self.data['ROA'] = self.data[('income_statement', 'net_income_loss')] / self.data[('balance_sheet', 'assets')]
        
#         #ROE
#         self.data['ROE'] = self.data[('income_statement', 'net_income_loss')] / self.data[('balance_sheet', 'equity')]
        
#         return self.data
    
#     def liquidity_ratios(self):
#         """ Calculates liquidity ratios."""
#         # data = self.self.data.copy()

#         #Current Ratio
#         self.data['current_ratio'] = self.data[('balance_sheet', 'current_assets')] / self.data[('balance_sheet', 'current_liabilities')]
        
#         #Quick Ratio
#         self.data['quick_ratio'] = (self.data[('balance_sheet', 'current_assets')] - self.data[('balance_sheet', 'inventory')]) / self.data[('balance_sheet', 'current_liabilities')]
        
    
#     def other_ratios(self):
#         """Calculates other ratios."""
#         # data = self.self.data.copy()
        
#         #Debt to Equity
#         self.data['debt_to_equity'] = self.data[('balance_sheet', 'liabilities')] / self.data[('balance_sheet', 'equity')]
        
#         #Interest Coverage
#         self.data['interest_coverage'] = self.data[('income_statement', 'operating_income_loss')] / self.data[('income_statement', 'interest_expense_operating')]
#         self.data['R&D_ratio'] = self.data[('income_statement', 'research_and_development')] / self.data[('income_statement', 'revenues')]
        
#         return self.data
    
#     def calculate_metrics(self):
#         """Orchestrates the calculation of financial metrics."""
#         self.profitability_ratios()
#         self.liquidity_ratios()
#         self.other_ratios()
        
#         return self.data
        

In [3]:
#Selecting some tickers to analyse
tickers = ['AAPL', 'MSFT']

In [4]:
#Fetching data from our SQL database

fetch_data = DataFetcher(tickers)
company_data = fetch_data.get_company_data()

In [5]:
#Pre Processing the data

prepocess = PreProcessingFinancials(company_data, SECTIONS, tickers)
data_dict, df = prepocess.preprocess_financials(GENERAL_COLUMNS)

In [6]:
#Calculating Financial ratios

metrics = CalculateMetrics(df)
final_data = metrics.calculate_metrics()

In [17]:
final_data

company_name start_date   end_date fiscal_period  \
ticker filing_date                                                       
AAPL   2024-08-05        APPLE INC 2023-06-30 2024-06-29           TTM   
       2024-08-02       Apple Inc. 2024-03-31 2024-06-29            Q3   
       2024-05-03       Apple Inc. 2023-12-31 2024-03-30            Q2   
       2024-02-02       Apple Inc. 2023-10-01 2023-12-30            Q1   
       2023-11-06       Apple Inc. 2023-07-02 2023-09-30            Q4   
...                            ...        ...        ...           ...   
MSFT   2010-10-28   MICROSOFT CORP 2010-07-01 2010-09-30            Q1   
       2010-08-06   MICROSOFT CORP 2010-04-01 2010-06-30            Q4   
       2010-01-28   MICROSOFT CORP 2009-10-01 2009-12-31            Q2   
       2009-10-23   MICROSOFT CORP 2009-07-01 2009-09-30            Q1   
       NaT          MICROSOFT CORP 2011-10-01 2011-12-30            Q2   

                    fiscal_year acceptance_datetime  timeframe tickers   sic  \
ticker filing_date                                                             
AAPL   2024-08-05           NaN                None        ttm    AAPL  3571   
       2024-08-02        2024.0                None  quarterly    AAPL  3571   
       2024-05-03        2024.0                None  quarterly    AAPL  3571   
       2024-02-02        2024.0                None  quarterly    AAPL  3571   
       2023-11-06        2023.0                None  quarterly    AAPL  3571   
...                         ...                 ...        ...     ...   ...   
MSFT   2010-10-28        2011.0                None  quarterly    MSFT  7372   
       2010-08-06        2010.0                None  quarterly    MSFT  7372   
       2010-01-28        2010.0                None  quarterly    MSFT  7372   
       2009-10-23        2010.0                None  quarterly    MSFT  7372   
       NaT               2012.0                None  quarterly    MSFT  7372   

                    (balance_sheet, noncurrent_liabilities)  ...  \
ticker filing_date                                           ...   
AAPL   2024-08-05                                  133280.0  ...   
       2024-08-02                                  133280.0  ...   
       2024-05-03                                  139395.0  ...   
       2024-02-02                                  145441.0  ...   
       2023-11-06                                  145129.0  ...   
...                                                     ...  ...   
MSFT   2010-10-28                                   18741.0  ...   
       2010-08-06                                   13791.0  ...   
       2010-01-28                                   12098.0  ...   
       2009-10-23                                   11639.0  ...   
       NaT                                              NaN  ...   

                    gross_margin  operating_margin  net_profit_margin  \
ticker filing_date                                                      
AAPL   2024-08-05       0.459620          0.312741           0.264407   
       2024-08-02       0.462572          0.295557           0.250044   
       2024-05-03       0.465781          0.307428           0.260443   
       2024-02-02       0.458750          0.337637           0.283638   
       2023-11-06       0.451708          0.301336           0.256497   
...                          ...               ...                ...   
MSFT   2010-10-28       0.806175          0.439395           0.334054   
       2010-08-06       0.802357          0.369724           0.281688   
       2010-01-28       0.809273          0.447534           0.350226   
       2009-10-23       0.780031          0.346904           0.276625   
       NaT              0.730045               NaN           0.317165   

                         ROA       ROE  current_ratio  quick_ratio  \
ticker filing_date                                                   
AAPL   2024-08-05   0.307456  1.528392     

In [7]:
#List with values we will be using in the dashboard dropdowns

tickers = final_data.index.get_level_values(0).unique()
timeframes = final_data['timeframe'].unique()
metrics = ['gross_margin',
           'operating_margin',
           'net_profit_margin',
           'ROA',
           'ROE',
           'current_ratio',
           'quick_ratio',
           'debt_to_equity',
           'interest_coverage',
           'R&D_ratio']
graph_types = ['Line', 'Bar', 'Scatter']



In [8]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import plotly.graph_objs as go
import pandas as pd
from dash.exceptions import PreventUpdate
from flask_caching import Cache

# Initialising the Dash app
# app = dash.Dash(__name__)
app = dash.Dash(__name__, suppress_callback_exceptions=True)

# caching to optimise
cache = Cache(app.server, config={'CACHE_TYPE': 'simple'})


# Layout
app.layout = html.Div([
    html.H1("Financial Dashboard"),

    # Dropdown to select the ticker
    dcc.Dropdown(
        id='select-ticker',
        options=[{'label': ticker, 'value': ticker} for ticker in tickers],
        value=tickers[0],  # default value set to the first ticker
        style={'width': '50%'}
    ),

    # Dropdown to select timespan
    dcc.Dropdown(
        id='select-timespan',
        options=[
            {'label': 'Minute', 'value': 'minute'},
            {'label': 'Hourly', 'value': 'hour'},
            {'label': 'Daily', 'value': 'daily'},
        ],
        value='daily',  # Default value
        style={'width': '50%'}
    ),
    


    # Tabs for organizing content
    dcc.Tabs(id='tabs', value='tab-financials', children=[
        dcc.Tab(label='Financial Ratios', value='tab-financials'),
        dcc.Tab(label='Stock Data', value='tab-stock')
    ]),
    html.Div(id='tabs-content')
])


In [9]:
@app.callback(
    Output('tabs-content', 'children'),
    [Input('tabs', 'value')]
)
def render_content(tab):
    if tab == 'tab-financials':
        return html.Div([
            # Financials tab content
            dcc.Dropdown(id='select-timeframe', options=[{'label': t, 'value': t} for t in timeframes],
                         value='quarterly', style={'width': '50%'}),
            dcc.Dropdown(id='select-metric', options=[{'label': m, 'value': m} for m in metrics],
                         value=metrics[0], style={'width': '50%'}),
            dcc.Dropdown(id='select-graph-type', options=[{'label': g, 'value': g} for g in graph_types],
                         value='Line', style={'width': '50%'}),
            dcc.Graph(id='financial-plot')
        ])
    elif tab == 'tab-stock':
        return html.Div([
            dcc.Dropdown(id='select-technical-indicator', options=[], value=None, multi = True, style={'width': '50%'}),
            dcc.Graph(id='stock-plot')
        ])
    return html.Div()  # Default empty content


In [10]:

@app.callback(
    Output('select-technical-indicator', 'options'),
    [Input('select-ticker', 'value'),
     Input('select-timespan', 'value'),
     Input('tabs', 'value')]  # Added Input to track the active tab
)
def update_indicator_options(selected_ticker, selected_timespan, tab):
    # Only update the technical indicators if the "Stock Data" tab is active
    if tab != 'tab-stock':
        raise PreventUpdate

    # Fetch and process data as usual
    data = get_cached_data(selected_ticker, selected_timespan)
    # non_indicator_columns = ['id', 'date', 'timestamp', 'ticker', 'open', 'high', 'low', 'close', 'volume', 'vwap', 'transactions']
    # indicator_options = [{'label': col, 'value': col} for col in data.columns if col not in non_indicator_columns]
    non_indicator_columns = ['id', 'date', 'timestamp', 'ticker']
    indicator_options = [{'label': col, 'value': col} for col in data.columns if col not in non_indicator_columns]
    return indicator_options

# Second Callback: Update Stock Plot Figure
@app.callback(
    Output('stock-plot', 'figure'),
    [Input('select-ticker', 'value'),
     Input('select-timespan', 'value'),
     Input('select-technical-indicator', 'value')]
)
def update_stock_graph(selected_ticker, selected_timespan, selected_indicators):
    data = get_cached_data(selected_ticker, selected_timespan)
    if not selected_indicators:
        # If no indicator is selected, use a default one or do not plot
        non_indicator_columns = ['id', 'date', 'timestamp', 'ticker']
        technical_indicators = [col for col in data.columns if col not in non_indicator_columns]
        selected_indicators = technical_indicators[:1] if technical_indicators else []
    
    fig = go.Figure()

    if selected_indicators:
        for indicator in selected_indicators:
            fig.add_trace(go.Scatter(
                x=data['date'],
                y=data[indicator],
                mode='lines',
                name=indicator
            ))
        indicators_str = ', '.join(selected_indicators)
        fig.update_layout(
            title=f"{indicators_str} for {selected_ticker} ({selected_timespan})"
        )
    else:
        fig.update_layout(title="No Technical Indicator Selected")
    
    return fig



In [11]:
@app.callback(
    Output('financial-plot', 'figure'),
    [Input('select-ticker', 'value'),
     Input('select-timeframe', 'value'),
     Input('select-metric', 'value'),
     Input('select-graph-type', 'value')]
)
def update_financial_graph(selected_ticker, selected_timeframe, selected_metric, selected_graph_type):
    # Fetch and process financial data
    # Assuming final_data is your preprocessed financial DataFrame

    # Filter the dataframe based on the selected ticker
    filtered_df = final_data[(final_data.index.get_level_values(0) == selected_ticker)]

    # Filter the dataframe based on the selected timeframe
    if selected_timeframe == 'quarterly':
        # Filtering only quarterly data
        filtered_df = filtered_df[filtered_df['timeframe'] == 'quarterly']
        # Use the 'period' column as the x-axis
        x_axis = filtered_df['period'].astype(str)
    else:
        # For FY or TTM, use fiscal_year as the x-axis
        filtered_df = filtered_df[filtered_df['timeframe'] == selected_timeframe]
        x_axis = filtered_df['fiscal_year']

    # Figure is based on the selected graph type
    if selected_graph_type == 'Line':
        fig = go.Figure(data=[
            go.Scatter(x=x_axis, y=filtered_df[selected_metric], mode='lines+markers')
        ])
    elif selected_graph_type == 'Bar':
        fig = go.Figure(data=[
            go.Bar(x=x_axis, y=filtered_df[selected_metric])
        ])
    elif selected_graph_type == 'Scatter':
        fig = go.Figure(data=[
            go.Scatter(x=x_axis, y=filtered_df[selected_metric], mode='markers')
        ])

    # Updating title and layout
    fig.update_layout(title=f"{selected_metric} for {selected_ticker} ({selected_timeframe})")
    return fig


In [12]:
# Caching function
@cache.memoize(timeout=300)  # Cache data for 5 minutes
def get_cached_data(ticker, timespan = 'daily'):
    # print(f"Fetching data for {ticker} with {timespan}")
    pipe = Pipeline([ticker])
    data = pipe.pipeline(combine=True, timespan=timespan)
    return data


In [13]:
if __name__ == '__main__':
    app.run_server(debug=True)
